# Apache Beam and Airflow for Pipelines

So far we've been looking at building ML pipeline componets separately and how they would interconnect. Now we should look at how we can put all those components together and how to run a full pipeline. To do that we will mainly look at 2 orchestors Apache Airflow and Beam.

The importance of this type of orchestration tool is that otherwise we would need to write code to check when a component complete its task and trigger the other etc. (earlier we used interactive pipelines, but we cant use those in production systems.)

### Apache Beam

TFX comes with a beam version by default. Therefore for a minimal setup using beam as orchastration tool is a valid choice. It is straight forward to setup and use while allowing to use existing data processing pipelines like google cloud dataflow.
But it lacks tools for scheduling model updates or monitoring the pipeline job progresses.


### Apache Airflow

Apache Airflow is already a widely used tool for data-loading task. It provide support to connect with production ready databases like PostgreSQL and execute partial pipelines etc which can save significant amount of time.


### Kubeflow pipelines

If there's an already existing kubernetes pipeline then it would make sense to use a Kubeflow pipelines tool. But it is complicated to setup compared to other tools. On the other hand it opens up opportunities to view TFDV and TFMA visializations, model lineage and artifact collections.


With those details in mind, we can now look in to converting our interactive code to a script which can be used to automate the whole process.

First we will define a function to initialize all the component required by our pipeline. This helps us to during the configuration of pipelines to setup for different orchastrators.

In [ ]:
### For demonstration only. DO NOT RUN!


import tensorflow_model_analysis as tfma
from tfx.components import (CsvExampleGen, Evaluator, ExampleValidator, Pusher,
                            ResolverNode, SchemaGen, StatisticsGen, Trainer,
                            Transform)
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.proto import pusher_pb2, trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing


def init_components(data_dir, module_file, serving_model_dir, training_steps, eval_steps):


    example_gen = CsvExampleGen(...)
    statistics_gen = StatisticsGen(...)
    schema_gen = SchemaGen(...)
    validator = ExampleValidator(...)
    transformer = Transform(...)
    trainer = Trainer(...)
    model_resolver = ResolverNode(...)
    eval_config = tfma.EvalConfig(...)
    evaluator = Evaluator(...)
    pusher = Pusher(...)

    components = [
        example_gen, statistics_gen, schema_gen, 
        validator, transformer, trainer, 
        model_resolver, evaluator, pusher
    ]
    return components



To this function we need to provide the inputs such as data location, where the model should be saved, module files required by the trainer and transform components along with hyperparameters.


When we are converting a simple interactive pipeline to a beam or airflow version we can directly use our jupyter notebooks. For any cells we dont need to export use the cell magic `%%skip_for_export`. Then we can setup the rest as follows.

In [ ]:
# DO NOT RUN! Dummy Code segment

orchester_type = 'beam'  # or airflow
pipeline_name  = 'test_pipeline_beam'

notebook_file = 'notebook/file/location'

#pipeline inputs
data_dir = 'data/stored/path'
module_file = 'module/file/stored/path'
requirement_file = 'requirement/config/file/path'

output_base_path = 'base/output/path'
serving_model_dir = 'model/save/dir/path'
pipeline_root = 'pipeline/base/path'
metadata_path = 'data/store/path'

pipeline_export_file = 'pipline/script/export/path'

# From interactive context of TFX run this
context.export_to_pipeline(notebook_file_path=notebook_file,
                           export_file_path=pipeline_export_file,
                           runner_type=orchester_type)

Above code will generate a python script that can be run using Apache beam or Airflow directly. Else we can go through the method descriobed below as well.

### Orchestrating using Apache Beam

Apache beam is not complex nor feature rich as airflow or kubeblow. But it is easy to use and is a good method to troubleshoot/debug our works. 

Below we have defined a Beam Pipeline which accept the TFX pipeline components as an argument and also connects to the SQLite database as the metadata store.

In [ ]:
import absl # used for advanced logging. https://abseil.io/docs/python/guides/logging
from tfx.orchestration import metadata, pipeline

def init_beam_pipe(components, pipeline_root, direct_num_workers):

    absl.logging.info(f'Pipeline root is set to {pipeline_root}.')

    beam_args = [f'--direct_num_workers={direct_num_workers}', 
                 f'--requirements_file={requirement_file}']

    p = pipeline.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=False, # If this is true some components will run using cached data
        metadata_connection_config=metadata.sqlite_metadata_connection_config(metadata_path),
        beam_pipeline_args=beam_args
    )

    return p

In [ ]:
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

components = init_components(...)

pipeline = init_beam_pipe(components, pipeline_root, direct_num_workers=2)
BeamDagRunner().run(pipeline)

Like that we can define a beam pipeline and run. Above is not a concrete code sample. But outline the gist of what needs to be done. Also we can scale up this pipeline using `Apache Flink` if needed.

### Orchesrating using Airflow

